In [1]:
import spcm
from spcm import units
import numpy as np

# card : spcm.Card

ip = 'TCPIP::192.168.1.83::INSTR'

card = spcm.Card(ip)

card.open(ip)

# setup card for DDS
card.card_mode(spcm.SPC_REP_STD_DDS)

# Setup the channels
channels = spcm.Channels(card)
print(len(channels))
channels.enable(True)
channels.output_load(50 * units.ohm)
channels.amp(1. * units.V)
card.write_setup()

# trigger mode
trigger = spcm.Trigger(card)
trigger.or_mask(spcm.SPC_TMASK_EXT0) # disable default software trigger
trigger.ext0_mode(spcm.SPC_TM_POS) # positive edge
trigger.ext0_level0(1.5 * units.V) # Trigger level is 1.5 V (1500 mV)
trigger.ext0_coupling(spcm.COUPLING_DC) # set DC coupling
card.write_setup()

# Setup DDS functionality
dds = spcm.DDS(card, channels=channels)
dds.reset()

dds.trg_src(spcm.SPCM_DDS_TRG_SRC_CARD)

core_list = [hex(2**n) for n in range(20)]

# core_list = [spcm.SPCM_DDS_CORE8,  spcm.SPCM_DDS_CORE9,  spcm.SPCM_DDS_CORE10, spcm.SPCM_DDS_CORE11, spcm.SPCM_DDS_CORE20]

# dds.cores_on_channel(1, *core_list)

card.start(spcm.M2CMD_CARD_ENABLETRIGGER)

2


In [2]:
# max_f = 1.2e6
max_f = .7e6
n_tweezers = 1

# f_list = [75.e6 - max_f,75.e6,75.e6 + max_f]
# f_list = [75.e6 - max_f,75.e6 + max_f]
f_list = np.linspace(75.e6 - max_f,75.e6 + max_f,n_tweezers)
# f_list = [75.e6]
# a_list = np.linspace(1/n_tweezers,1/n_tweezers,n_tweezers)
a_list = [.5,.5]

for tweezer_idx in range(len(core_list)):
    if tweezer_idx < len(f_list):
        dds[tweezer_idx].amp(a_list[tweezer_idx])
        dds[tweezer_idx].freq(f_list[tweezer_idx])
    else:
        dds[tweezer_idx].amp(0.)
dds.exec_at_trg()
dds.write_to_card()

# Start command including enable of trigger engine

# input("Press Enter to Exit")

In [2]:
def add_tweezers_to_buffer(n_tweezers, freq_spacing=0.7e6 * 2, amp_list=[]):
    max_f = freq_spacing
    n_tweezers = n_tweezers
    amp_list = [.2,.215]

    # f_list = [75.e6 - max_f,75.e6,75.e6 + max_f]
    # f_list = [75.e6 - max_f,75.e6 + max_f]
    f_list = np.linspace(75.e6 - (n_tweezers-1)/2 * max_f,
                         75.e6 + (n_tweezers-1)/2 * max_f,
                         n_tweezers)
    
    if not amp_list:
        # a_list = np.linspace(1/n_tweezers,1/n_tweezers,n_tweezers)
        a_list = [0.2]*n_tweezers
    else:
        a_list = amp_list

    for tweezer_idx in range(len(core_list)):
        if tweezer_idx < len(f_list):
            if isinstance(a_list,list):
                dds[tweezer_idx].amp(a_list[tweezer_idx])
            else:
                dds[tweezer_idx].amp(a_list)
            dds[tweezer_idx].freq(f_list[tweezer_idx])
        else:
            dds[tweezer_idx].amp(0.)
    dds.exec_at_trg()
    dds.write_to_card()

add_tweezers_to_buffer(1, amp_list=0.2, freq_spacing = 1.e6)

In [3]:
# card.close(card._handle)
card.stop()

card.close(card._handle)